# Evolver Loop 5 Analysis

## Situation Summary
- 5 experiments completed, ALL at score 70.734327 (same as baseline)
- Target: 68.931058 (need 1.8 points improvement, ~2.5%)
- The ensemble with overlaps scores 67.77 but overlaps cannot be repaired
- All local optimization approaches have failed

## Key Questions
1. What are the best public kernels achieving?
2. What datasets/CSVs are available that we haven't tried?
3. Is there a fundamentally different approach we're missing?

In [ ]:
import numpy as np
import pandas as pd
import os
import glob
from numba import njit
import math

os.chdir('/home/code')
print('Working directory:', os.getcwd())

In [ ]:
# Tree polygon template
@njit
def make_polygon_template():
    tw=0.15; th=0.2; bw=0.7; mw=0.4; ow=0.25
    tip=0.8; t1=0.5; t2=0.25; base=0.0; tbot=-th
    x=np.array([0,ow/2,ow/4,mw/2,mw/4,bw/2,tw/2,tw/2,-tw/2,-tw/2,-bw/2,-mw/4,-mw/2,-ow/4,-ow/2],np.float64)
    y=np.array([tip,t1,t1,t2,t2,base,base,tbot,tbot,base,base,t2,t2,t1,t1],np.float64)
    return x, y

@njit
def score_group(xs, ys, degs, tx, ty):
    n = xs.size
    V = tx.size
    mnx = 1e300; mny = 1e300; mxx = -1e300; mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c = math.cos(r)
        s = math.sin(r)
        xi = xs[i]
        yi = ys[i]
        for j in range(V):
            X = c * tx[j] - s * ty[j] + xi
            Y = s * tx[j] + c * ty[j] + yi
            if X < mnx: mnx = X
            if X > mxx: mxx = X
            if Y < mny: mny = Y
            if Y > mxy: mxy = Y
    side = max(mxx - mnx, mxy - mny)
    return side * side / n

tx, ty = make_polygon_template()
print('Template created')

In [ ]:
# Load and score a submission
def strip(a):
    return np.array([float(str(v).replace('s', '')) for v in a], np.float64)

def load_and_score(filepath):
    try:
        df = pd.read_csv(filepath)
        if not {'id', 'x', 'y', 'deg'}.issubset(df.columns):
            return None
        df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)
        total = 0.0
        for n, g in df.groupby('N'):
            if n < 1 or n > 200:
                continue
            xs = strip(g['x'].to_numpy())
            ys = strip(g['y'].to_numpy())
            ds = strip(g['deg'].to_numpy())
            total += score_group(xs, ys, ds, tx, ty)
        return total
    except:
        return None

print('Functions defined')

In [ ]:
# Check all available CSV files in snapshots
print('Searching for CSV files in snapshots...')

all_csvs = []
for root, dirs, files in os.walk('/home/nonroot/snapshots'):
    for f in files:
        if f.endswith('.csv'):
            all_csvs.append(os.path.join(root, f))

print(f'Found {len(all_csvs)} CSV files')

# Score each file
scores = []
for fp in all_csvs:
    score = load_and_score(fp)
    if score is not None:
        scores.append((score, fp))

scores.sort()
print(f'\nTop 20 best scoring CSVs:')
for score, fp in scores[:20]:
    print(f'  {score:.6f}: {fp.split("/")[-1]}')

print(f'\nBaseline: 70.734327')
print(f'Target: 68.931058')

In [ ]:
# Check the jonathanchan kernel's approach
# It uses ensemble + SA + fractional translation
# Let's see what datasets it references

print('Datasets referenced by jonathanchan kernel:')
print('1. bucket-of-chump')
print('2. santa25-public')
print('3. telegram-public-shared-solution-for-santa-2025')
print('4. santa-2025-try3')
print('5. Various kernel outputs')

# Check if we have these datasets
print('\nChecking available datasets...')
for name in ['bucket-of-chump', 'santa25-public', 'telegram-public', 'santa-2025-try3']:
    found = [f for f in all_csvs if name in f.lower()]
    print(f'{name}: {len(found)} files found')

In [ ]:
# The key insight from the jonathanchan kernel:
# 1. It ensembles MANY sources (20+ different notebooks/datasets)
# 2. It runs a C++ SA optimizer (sa_v1_parallel.cpp) with:
#    - 15000 iterations
#    - 5 restarts
#    - Population-based search (keeps top 3)
#    - Fractional translation post-processing
#    - Backward propagation

# The egortrushin kernel uses a different approach:
# - SA with translations (structured packing)
# - 2 base trees at different angles
# - Translate in grid pattern to create N trees
# - Works well for large N (>50)

print('Key approaches from top kernels:')
print('1. jonathanchan: Ensemble + C++ SA + fractional translation')
print('2. egortrushin: SA with translations (structured packing)')
print('3. saspav: bbox3 binary + fix_direction')
print('4. smartmanoj: C++ tree_packer with OpenMP')

In [ ]:
# Check the baseline submission
baseline_path = '/home/nonroot/snapshots/santa-2025/21105319338/code/datasets/santa-2025-csv/santa-2025.csv'
baseline_score = load_and_score(baseline_path)
print(f'Baseline score: {baseline_score:.6f}')

# Check if there are any better valid submissions
better_valid = [(s, f) for s, f in scores if s < baseline_score - 0.001]
print(f'\nCSVs with better score than baseline: {len(better_valid)}')
for s, f in better_valid[:10]:
    print(f'  {s:.6f}: {f.split("/")[-1]}')

In [ ]:
# The problem: ALL better-scoring CSVs have overlaps
# The baseline is already at a local optimum

# What we need to try:
# 1. Run the jonathanchan C++ optimizer with MUCH higher parameters
# 2. Try the egortrushin structured packing approach properly
# 3. Look for external datasets that might have valid improvements

# Let's check the C++ optimizer we have
print('Checking C++ optimizer...')
import subprocess
result = subprocess.run(['ls', '-la', '/home/code/experiments/'], capture_output=True, text=True)
print(result.stdout)

In [ ]:
# Check if sa_v3_parallel is compiled and working
sa_binary = '/home/code/experiments/sa_v3_parallel'
if os.path.exists(sa_binary):
    print('sa_v3_parallel binary exists')
    # Check if it runs
    result = subprocess.run([sa_binary, '-h'], capture_output=True, text=True)
    print('Help output:', result.stdout[:500] if result.stdout else 'No output')
    print('Error:', result.stderr[:500] if result.stderr else 'No error')
else:
    print('sa_v3_parallel binary not found - need to compile')

In [ ]:
# CRITICAL INSIGHT:
# The jonathanchan kernel runs the C++ optimizer in an ENDLESS LOOP
# with generation-based optimization until no improvement for 3 generations
# This is different from our single-pass approach!

# Key parameters from jonathanchan:
# - si = 20000 (iterations)
# - nr = 80 (restarts)
# - max_retries = 3 (generations without improvement before stopping)
# - max_retry_retries = 3

# Our approach has been:
# - si = 15000 (iterations)
# - nr = 5-6 (restarts)
# - Single pass only

print('CRITICAL DIFFERENCE:')
print('jonathanchan: 20000 iterations, 80 restarts, multiple generations')
print('Our approach: 15000 iterations, 5-6 restarts, single pass')
print('')
print('We need to run with MUCH higher parameters and multiple generations!')

In [ ]:
# Summary of what to try next:
print('=== RECOMMENDED NEXT STEPS ===')
print('')
print('1. COMPILE AND RUN C++ OPTIMIZER WITH HIGH PARAMETERS:')
print('   - Iterations: 30000-50000')
print('   - Restarts: 32-80')
print('   - Multiple generations (keep running until no improvement)')
print('')
print('2. TRY EGORTRUSHIN STRUCTURED PACKING:')
print('   - For large N (>50), use grid-based translations')
print('   - 2 base trees at different angles')
print('   - SA to optimize translation parameters')
print('')
print('3. ENSEMBLE FROM MULTIPLE SOURCES:')
print('   - Combine best per-N from all available CSVs')
print('   - Run C++ optimizer on the ensemble')
print('')
print('4. FOCUS ON SPECIFIC N VALUES:')
print('   - Identify N values with most room for improvement')
print('   - Target optimization on those specific N values')